In [ ]:
## dont run beyond 60 lack of computation - 92% ram 
## try 40 next

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import pathlib
import PIL
import os
import tarfile
import cv2
import numpy as np
import time

lower_age_limit = 1
upper_age_limit = 30

In [ ]:
tar_files = ['part1.tar.gz', 'part2.tar.gz', 'part3.tar.gz']
output_folder = os.path.join(os.getcwd(), 'all_images')

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

    for tar_file in tar_files:
        with tarfile.open(tar_file, 'r:gz') as tar:
            for member in tar.getmembers():
                # Remove the leading directory name from the member's name
                member.name = os.path.basename(member.name)
                tar.extract(member, output_folder)

In [ ]:
data_dir = pathlib.Path('.//all_images')

In [ ]:
image_count = len(list(data_dir.glob('**/*.jpg')))
print(image_count)

In [ ]:
# age_1 = list(data_dir.glob('1_*'))
# PIL.Image.open(str(age_1[1]))

In [ ]:
age_images_dict = {}
ages_labels_dict = {}

for i in range(lower_age_limit, upper_age_limit):
    age_images_dict[i] = list(data_dir.glob(f'{i}_*'))
    ages_labels_dict[i] = i-1

# some ages are missing 

In [ ]:
# img = cv2.imread(str(age_images_dict[3][0]))
# img.shape

In [ ]:
# start = time.time()
# x, y = [], []

# for age_number, images in age_images_dict.items():
#     for image in images:
#         try:
#             img = cv2.imread(str(image))
#             resized_img = cv2.resize(img,(180,180))
#         except cv2.error as e:
#             print(f"Error processing {image}: {str(e)}")
#             continue
#         x.append(resized_img)
#         y.append(ages_labels_dict[age_number])
        
# x = np.array(x)/255
# y = np.array(y)
# print(time.time()-start)

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
def process_image(image):
    try:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(180,180))
        return resized_img
    except cv2.error as e:
        print(f"Error processing {image}: {str(e)}")
        return None

x, y = [], []

with ThreadPoolExecutor() as executor:
    futures = []
    for age_number, images in age_images_dict.items():
        for image in images:
            future = executor.submit(process_image, image)
            futures.append((future, age_number))

    for future, age_number in futures:
        result = future.result()
        if result is not None:
            x.append(result)
            y.append(ages_labels_dict[age_number])

x = np.array(x)
y = np.array(y)

In [ ]:
np.savez( os.getcwd() + '/ age_image_data.npz', x, y)
del x, y, age_images_dict, ages_labels_dict, futures, result, images

# # run from here if saved .npz file

In [ ]:
import numpy as np
import tensorflow as tf
import os

from tensorflow.keras import datasets, layers, models
from tensorflow import keras
from tensorflow.keras.models import Sequential

In [ ]:
with np.load(os.getcwd() + '/ age_image_data.npz', mmap_mode='c') as data:
    x = data['arr_0']
    y = data['arr_1']

num_classes = len(np.unique(y))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=0)
del x, y

In [ ]:
data_augmentation = keras.Sequential(
    [
        tf.keras.layers.Rescaling(scale = 1./255, offset=0.0),
        layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(180, 180, 3)),
        layers.experimental.preprocessing.RandomRotation(0.1),
        layers.experimental.preprocessing.RandomZoom(0.1)    
    ]
)

model = Sequential([
    
    data_augmentation,
    
    layers.Conv2D(16, 3, padding='same', activation='relu', input_shape = (180,180,3)),
    layers.MaxPooling2D(),
    
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
              
model.fit(X_train, y_train, epochs=10)
del X_train, y_train

In [ ]:
model.evaluate(X_test,y_test)
del X_test,y_test